In [1]:
#Created by: Georgina Hoagland
#Created: Aug 29, 2016
#Last Modified: October 20, 2016

#To check: Day counters & consistency
#To add: Units

from QuantLib import *

#setup global eval date 
calc_date = Date(20, 10, 2016)
Settings.instance().evaluationDate = calc_date

#Calendar, rules, conventions

calendars = [Argentina, Australia, BespokeCalendar, Brazil, Canada, China, \
             CzechRepublic, Denmark, Finland, Germany, HongKong, Hungary, Iceland, \
             India, Indonesia, Israel, Italy, Japan, JointCalendar, Mexico, NewZealand,\
             Norway, NullCalendar, Poland, Romania, Russia, SaudiArabia, Singapore, \
             Slovakia, SouthAfrica, SouthKorea, Sweden, Switzerland, Taiwan, TARGET, \
             Turkey, Ukraine, UnitedKingdom, UnitedStates, WeekendsOnly]

bus_day_conventions = [Following, ModifiedFollowing, Preceding, ModifiedPreceding, \
                       Unadjusted,HalfMonthModifiedFollowing]

gen_rules = [DateGeneration.Backward, DateGeneration.Forward, DateGeneration.Zero, \
             DateGeneration.ThirdWednesday] 



In [13]:
#Swap info

settlementDays = 2
    
fixedLegNotional = 100 #dollars
fixedLegTenor = Period(6, Months)
fixedLegFrequency = Semiannual
fixedLegAdjustment = ModifiedFollowing
fixedLegDayCounter = Thirty360()

floatingLegSpread = 0.0 #Libor rate - Treasury security of same maturity
floatingLegTenor = Period(6, Months) #Libor 6M
floatingLegFrequency = Semiannual

#these are created using the index conventions 
#floatingLegAdjustment = ModifiedFollowing
#floatingLegDayCounter = Thirty360() 

# term structure handles
discountTermStructure = RelinkableYieldTermStructureHandle()
forwardTermStructure = RelinkableYieldTermStructureHandle() 
# RelinkableHandle<YieldTermStructure>

#need to change to be able to change currency?
liborIndex = USDLibor(floatingLegTenor, discountTermStructure)

floatingLegDayCounter = liborIndex.dayCounter()
floatingLegAdjustment = liborIndex.businessDayConvention()


#use index in from lists to choose calendar, convention, etc.
eval_date = calc_date
cal = calendars[-6]() #TARGET

gen_rule = gen_rules[0] # Backward 
end_of_month = True

settlementDate = cal.advance(eval_date, settlementDays, Days, floatingLegAdjustment)

#Unadjusted calendar for termination purposes
termination_cal = calendars[-18]() #NullCalendar
termination_convention = bus_day_conventions[4] #Unadjusted

In [14]:
#Inputs

ois = {   (1,Days): 0.004,
          (1,Weeks): 0.004,
          (2,Weeks): 0.004,
          (3,Weeks): 0.00417,
          (1,Months): 0.004175,
          (2,Months): 0.00429,
          (3,Months): 0.004366,
          (4,Months): 0.004515,
          (5,Months): 0.00472,
          (6,Months): 0.00485,
          (9,Months): 0.00518,
          (12,Months): 0.005445,
          (18,Months): 0.005801,
          (2,Years): 0.006070,
          (3,Years): 0.006560,
          (4,Years): 0.007010,
          (5,Years): 0.007490,
          (7,Years): 0.012512,
          (10,Years): 0.013925,
          (12,Years): 0.014701,
          (15,Years): 0.015529,
          (20,Years): 0.016385,
          (25,Years): 0.016780,
          (30,Years): 0.016989,
          (40,Years): 0.016995,
          (50,Years): 0.017085}

#convert to ql Quote objects
for n,unit in ois.keys():
    ois[(n,unit)] = SimpleQuote(ois[(n,unit)])
    
#Should probably be using a different index but not sure which one?
#needed for oisHelpers
overnightIndex = FedFunds(discountTermStructure)

swaps = { (1,Years): 0.00900,
          (2,Years): 0.00975,    
          (3,Years): 0.01040,
          (4,Years): 0.01099,
          (5,Years): 0.01160,
          (6,Years): 0.01224,
          (7,Years): 0.01284,
          (8,Years): 0.01343,
          (9,Years): 0.01390,
          (10,Years): 0.01438,
          (15,Years): 0.01610,
          (20,Years): 0.01707,
          (25,Years): 0.01752,
          (30,Years): 0.01778} # 1 Year = 90bps, etc. 


#convert swap quotes from floats to Quote objects
for (n,unit) in swaps.keys():
    swaps[(n,unit)] = SimpleQuote(swaps[(n,unit)])


In [15]:
#Schedule

#creates dicts with keys of year term and values of termination date/payment schedule 
termination_date = {}
for i in range(1, 31):
    termination_date[i] = termination_cal.advance(settlementDate, Period(i, Years), \
                                                  termination_convention)
fixedSchedule = {}
for i in range(1, 31):
    fixedSchedule[i] = Schedule(settlementDate, termination_date[i], fixedLegTenor, cal, \
                                   fixedLegAdjustment, fixedLegAdjustment, gen_rule, \
                                   end_of_month)
floatingSchedule = {}
for i in range(1, 31):
    floatingSchedule[i] = Schedule(settlementDate, termination_date[i], floatingLegTenor, \
                                   cal, floatingLegAdjustment, floatingLegAdjustment, \
                                   gen_rule, end_of_month)

In [16]:
#Bootstrapping 

swapHelpers =  [SwapRateHelper(QuoteHandle(swaps[(n,unit)]), Period(n,unit), cal,\
                                     fixedLegFrequency, fixedLegAdjustment, \
                                     fixedLegDayCounter, liborIndex)\
                      for (n, unit) in swaps.keys() ]

oisHelpers = [OISRateHelper(settlementDays, Period (n, unit), QuoteHandle(ois[(n, unit)]), \
                            overnightIndex)\
                     for (n, unit) in ois.keys()]

#PiecewiseYieldCurve not exported to python
#currently using PiecewiseLogCubic
#corresponds to PiecewiseYieldCurve<Discount,LogCubic> in C++
#check day counter - using fixedLegDayCounter

piecewiseYieldCurve = PiecewiseLogCubicDiscount(settlementDays, cal, swapHelpers, \
                                           fixedLegDayCounter)
oisCurve = PiecewiseLogCubicDiscount(settlementDays, cal, oisHelpers, fixedLegDayCounter)
oisCurve.enableExtrapolation() #check if necessary

In [17]:
#Swap additions - linear
#uses averages to assign rates to non-existent swap maturities, i.e. 11Y, 12Y, 23Y
swapsAdditionsLinear = {}
for i in range(11, 31):
    if i % 5 != 0:
        swapsAdditionsLinear [(i, Years)] = \
                        ((swaps[((i//5 + 1) * 5, Years)].value() \
                          - swaps[((i//5)*5, Years)].value()) \
                          *(i % 5)/5) + swaps[((i//5)*5, Years)].value()
for (i, unit) in swapsAdditionsLinear.keys():
    swapsAdditionsLinear[(i, unit)] = SimpleQuote(\
                                            swapsAdditionsLinear[(i, unit)])
            
#both the given swap rates and intermediate values for non-existant maturities        
swapsAll = {}
for (i, unit) in swaps.keys():
    swapsAll[(i, unit)] = swaps[(i, unit)]
for (i, unit) in swapsAdditionsLinear.keys():
    swapsAll[(i, unit)] = swapsAdditionsLinear[(i, unit)]

In [18]:
#Swap Pricing

swapTypes = [VanillaSwap.Payer, VanillaSwap.Receiver]
swapType = swapTypes[0]

#discountTermStructure.linkTo(piecewiseYieldCurve)
forwardTermStructure.linkTo(piecewiseYieldCurve)
discountTermStructure.linkTo(oisCurve)
#forwardTermStructure.linkTo(oisCurve)
swapEngine = DiscountingSwapEngine(discountTermStructure)

    
vanillaSwaps = {}
for i in range(1, 31):
    vanillaSwaps[(i, Years)] = VanillaSwap(swapType, fixedLegNotional, fixedSchedule[i],\
                        swapsAll[(i, Years)].value(), fixedLegDayCounter, \
                        floatingSchedule[i], liborIndex, floatingLegSpread,\
                        floatingLegDayCounter)
    
    vanillaSwaps[(i, Years)].setPricingEngine(swapEngine)
    
swapsNPV = {}
for i in range(1, 31):
    swapsNPV[(i, Years)] = vanillaSwaps[(i, Years)].NPV()
    print i, swapsNPV[(i, Years)]

1 -0.347397179097
2 -0.715941220148
3 -1.1160527828
4 -1.53585509147
5 -1.97257716795
6 -1.22968329565
7 -0.12982775117
8 0.204634308238
9 0.034657714142
10 -0.287462472929
11 -0.300075884659
12 -0.23284202677
13 -0.25411907279
14 -0.371787111948
15 -0.554333397443
16 -0.562707989536
17 -0.593284971944
18 -0.652505620841
19 -0.749215428813
20 -0.891001201178
21 -0.894953194718
22 -0.931681238228
23 -0.994144599917
24 -1.07678990947
25 -1.17490941861
26 -1.20429079657
27 -1.24368725335
28 -1.29872390305
29 -1.37235055186
30 -1.46889421672


In [8]:
#Swaption Volatility

#using constant value of 40%
volatilityFlat = 0.40

volatilityStructure = ConstantSwaptionVolatility(settlementDays, cal,\
                                                     fixedLegAdjustment, volatilityFlat, \
                                                     fixedLegDayCounter)
# default: VolatilityType type=ShiftedLognormal, const Real shift=0.0

volatilityStructureHandle = SwaptionVolatilityStructureHandle(volatilityStructure)


In [9]:
#Swaption Inputs

swaptionMaturity = Period(3, Months)
swaptionExpiry = cal.advance(settlementDate, swaptionMaturity, floatingLegAdjustment)
settlementType = Settlement.Physical

#European
swaptionExercise = EuropeanExercise(swaptionExpiry)

In [10]:
#Swaption Object

swaptionPricingEngine = BlackSwaptionEngine(forwardTermStructure, volatilityStructureHandle)

swaptions = {}
for i in range(1, 31):
    swaptions [(i, Years)] = Swaption(vanillaSwaps[(i, Years)], swaptionExercise, \
                                      settlementType)
    
    swaptions[(i, Years)].setPricingEngine(swaptionPricingEngine)

In [11]:
#Swaption Pricing

swaptionsNPV = {}
for maturity in swaptions.keys():
    swaptionsNPV[maturity] = swaptions[maturity].NPV()
    
for i in range(1, 31):
    print i, swaptionsNPV[(i, Years)]


1 0.000325303719002
2 0.00109244019429
3 0.00211726237371
4 0.00354013488569
5 0.00552482242668
6 0.120994292129
7 0.624789459791
8 0.935436638775
9 0.966404550443
10 0.93152637443
11 1.05048696634
12 1.21265959693
13 1.33289494538
14 1.40673173536
15 1.45187299337
16 1.56875460076
17 1.67463871769
18 1.76710917846
19 1.84170403057
20 1.89520911817
21 2.00120389451
22 2.08988103162
23 2.16541230709
24 2.23082025883
25 2.28823133543
26 2.37121207005
27 2.44880013953
28 2.51692190196
29 2.57568463502
30 2.62270900303


In [12]:
#Implied Volatility

swaptionHelpers = {}
impliedVols = {}
for i in range(1, 31):
    swaptionHelpers[(i, Years)] = SwaptionHelper(swaptionMaturity, Period(i, Years), \
                                  QuoteHandle(SimpleQuote(volatilityFlat)), \
                                  liborIndex, fixedLegTenor, fixedLegDayCounter, \
                                  floatingLegDayCounter, forwardTermStructure)

    swaptionHelpers[(i, Years)].setPricingEngine(swaptionPricingEngine)

#    impliedVols[(i, unit)] = swaptionHelpers[(i, unit)].impliedVolatility(\
#                                                            swaptionsNPV[(i, unit)], \
#                                                            1e-7, 50, 0.0, 10.0)\
                             
#input values my be causing runtime errors in the implied vol value                